In [1]:
#******************************************
#** Data Boocamp January 2024 cohort     **
#**                                      **
#** Project #3: San Francisco Crime Data **
#**                                      **
#** M. Bowman, B. Lee, S. Kikuchi, M. Dai**
#** J. Wang, J. Bein                     **
#**                                      **
#******************************************


#*******************************************
#** This file is intended to be downloaded**
#** by each member of the group. The code **
#** is in two parts:                      **
#** Part #1: The data is imported, cleaned**
#** and prepared for exporting to a       **
#** MongoDB.                              **
#**                                       **
#**                                       **
#** Part #2: A local MongoDB will be      **
#** created, and a collection added to    **
#** that database. Lastly, the source data**
#** will be imported into the db.         **
#**                                       **
#*******************************************

#***************************
#** last edited 23MAY2024 **
#***************************

In [2]:
#*****************************
#** import the dependencies **
#*****************************

#uncomment the install line if you don't have PyMongo installed on your device.
!pip install pymongo 
from pymongo import MongoClient
import pandas as pd               #this will be needed to manipulate dataframes
import os                         #this module used to create file paths across operating systems
import csv                        #this module is used for reading CSV files

In [3]:
#******************************
#** PART ONE: DATA WRANGLING **
#******************************

In [4]:
#**************************************************
#** read the source data into a Pandas dataframe **
#**************************************************

csvpath = os.path.join("Police_Department_Incident_Reports.csv")    #a variable to hold the path to the CSV file
crime_df = pd.read_csv(csvpath)      #the dataframe holding the original (sample) set of data

# Display the first few rows of the DataFrame
print(crime_df.head())

        Incident Datetime Incident Date Incident Time  Incident Year  \
0  2023/03/13 11:41:00 PM    2023/03/13         23:41           2023   
1  2023/03/01 05:02:00 AM    2023/03/01         05:02           2023   
2  2023/03/13 01:16:00 PM    2023/03/13         13:16           2023   
3  2023/03/13 10:59:00 AM    2023/03/13         10:59           2023   
4  2023/03/14 06:44:00 PM    2023/03/14         18:44           2023   

  Incident Day of Week         Report Datetime        Row ID  Incident ID  \
0               Monday  2023/03/13 11:41:00 PM  125373607041      1253736   
1            Wednesday  2023/03/11 03:40:00 PM  125379506374      1253795   
2               Monday  2023/03/13 01:17:00 PM  125357107041      1253571   
3               Monday  2023/03/13 11:00:00 AM  125355107041      1253551   
4              Tuesday  2023/03/14 06:45:00 PM  125402407041      1254024   

   Incident Number  CAD Number  ... Longitude Point Neighborhoods  \
0        230167874         NaN  ...

In [5]:
#**************************************************
# check that the data was imported successfully  **
#**************************************************

num_rows = len(crime_df)
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 850895


In [6]:
#*******************************************
#** determine the datatypes in each column**
#*******************************************
print(crime_df.dtypes)

Incident Datetime                                        object
Incident Date                                            object
Incident Time                                            object
Incident Year                                             int64
Incident Day of Week                                     object
Report Datetime                                          object
Row ID                                                    int64
Incident ID                                               int64
Incident Number                                           int64
CAD Number                                              float64
Report Type Code                                         object
Report Type Description                                  object
Filed Online                                             object
Incident Code                                             int64
Incident Category                                        object
Incident Subcategory                    

This section describes the columns of data. Descriptions were obtained from
https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783/about_data

Incident Datetime                       The date and time when the incident occurred.

Incident Date                           The date the incident occured.

Incident Time                           The time the incident occured.

Incident Year                           The year the incident occured, provided as a convenience for filtering.

Incident Day of Week                    The day of the week the incident occured.

Report Datetime                         Distinct from Incident Datetime, Report Datetime is when the report was filed.

Row ID                                  A unique identifier for each row of data in the dataset.

Incident ID	                            This is the system generated identifier for incident reports. Incident IDs and Incident Numbers both                                                uniquely identify reports, but Incident Numbers are used when referencing cases and report documents.

Incident Number                         The number issued on the report, sometimes interchangeably referred to as the Case Number. This number is
                                        used to reference cases and report documents.
                                        
CAD Number                              The Computer Aided Dispatch (CAD) is the system used by the Department of Emergency Management (DEM) to
                                        dispatch officers and other public safety personnel. CAD Numbers are assigned by the DEM system and
                                        linked to relevant incident reports (Incident Number). Not all Incidents will have a CAD Number. Those
                                        filed online via Coplogic (refer to “Filed Online” field) and others not filed through the DEM system
                                        will not have CAD Numbers.
                                        
Report Type Code                        A system code for report types, these have corresponding descriptions within the dataset.

Report Type Description                 The description of the report type, can be one of: Initial; Initial Supplement; Vehicle Initial; 
                                        Vehicle Supplement; Coplogic Initial; Coplogic Supplement
                                        
Filed Online                            Non- emergency police reports can be filed online by members of the public using SFPD’s self-service
                                        reporting system called Coplogic Values in this field will be “TRUE” if Coplogic was used to file the 
                                        report. Please reference the link below for additional info: (http://sanfranciscopolice.org/reports).
                                        
Incident Code	                        Incident Codes are the system codes to describe a type of incident. A single incident report can have
                                        one or more incident types associated. In those cases you will see multiple rows representing a unique
                                        combination of the Incident ID and Incident Code.
                                        
Incident Category                       A category mapped on to the Incident Code used in statistics and reporting. Mappings provided by the
                                        Crime Analysis Unit of the Police Department.
                                        
Incident Subcategory                    A subcategory mapped to the Incident Code that is used for statistics and reporting. Mappings are
                                        provided by the Crime Analysis Unit of the Police Department.

Incident Description	                The description of the incident that corresponds with the Incident Code. These are generally 
                                        self-explanatory.

Resolution                              The resolution of the incident at the time of the report. Can be one of: • Cite or Arrest Adult • Cite or
                                        Arrest Juvenile* • Exceptional Adult • Exceptional Juvenile* • Open or Active • Unfounded Note: once a
                                        report is filed, the Resolution will not change. Status changes and/or updates must be provided using a
                                        Supplemental Report *Incidents identifying juvenile information are not included in this dataset.
                                        Please see the Juvenile Data section for more information.
                                
Intersection                            The 2 or more street names that intersect closest to the original incident separated by a backward slash 
                                        (\). Note, the possible intersections will only include those that satisfy the privacy controls.
                                     
CNN                                     The unique identifier of the intersection for reference back to other related basemap datasets. For more on
                                        the Centerline Node Network see 
                                        https://datasf.gitbook.io/draft-publishing-standards/standard-reference-data/basemap/street-centerlines-                                                                         nodes

Police District	                        The Police District where the incident occurred. District boundaries can be reviewed in the link below.
                                        Please note this field is entered by officers and not based on the point.
                                        Reference here: https://data.sfgov.org/d/wkhw-cjsf

Analysis Neighborhood                   This field is used to identify the neighborhood where each incident occurs. Neighborhoods and boundaries
                                        are defined by the Department of Public Health and the Mayor's Office of Housing and Community Development.
                                        Please reference the link below for additional info: https://data.sfgov.org/d/p5b7-5n3h Please note this
                                        boundary is assigned based on the intersection, it may differ from the boundary the incident actually                                                                            occurred within.

Supervisor District	                    Current Supervisor District: There are 11 members elected to the Board of Supervisors in San Francisco,
                                        each representing a geographic district. The Board of Supervisors is the legislative body for 
                                        San Francisco. The districts are numbered 1 through 11. Please reference the link below for additional
                                        info: https://data.sfgov.org/d/cqbw-m5m3 Please note this boundary is assigned based on the intersection,
                                        it may differ from the boundary the incident actually occurred within.

Supervisor District 2012                Previous 2012-2022 Supervisor District: There are 11 members elected to the Board of Supervisors in
                                        San Francisco, each representing a geographic district. The Board of Supervisors is the legislative body
                                        for San Francisco. The districts are numbered 1 through 11. Please reference the link below for additional
                                        info: https://data.sfgov.org/d/keex-zmn4 Please note this boundary is assigned based on the intersection,
                                        it may differ from the boundary the incident actually occurred within.

Latitude                                The latitude coordinate in WGS84, spatial reference is EPSG:4326

Longitude                               The longitude coordinate in WGS84, spatial reference is EPSG:4326

Point                                   Geolocation in OGC WKT format (e.g, POINT(37.4,-122.3)


The following fields were not defined on the website. TBD
Point                                                    
Neighborhoods                                           
ESNCAG - Boundary File                                  
Central Market/Tenderloin Boundary Polygon - Updated    
Civic Center Harm Reduction Project Boundary            
HSOC Zones as of 2018-06-05                             
Invest In Neighborhoods (IIN) Areas                     
Current Supervisor Districts                            
Current Police Districts                                


In [7]:
#***************************
#** drop unneeded columns: *
#***************************

print("Original DataFrame:")
print("-------------------")
print(crime_df.dtypes)

columns_to_drop = ['Report Datetime',
                   'Row ID',
                   'Incident ID',
                   'Incident Number',
                   'CAD Number',
                   'Report Type Code',
                   'Report Type Description',
                   'Filed Online',
                   'Incident Code',
                   'Intersection',
                   'CNN',
                   'Police District',
                   'Supervisor District',
                   'Supervisor District 2012',
                   'Point',
                   'ESNCAG - Boundary File',
                   'Central Market/Tenderloin Boundary Polygon - Updated',
                   'Civic Center Harm Reduction Project Boundary',
                   'HSOC Zones as of 2018-06-05',
                   'Invest In Neighborhoods (IIN) Areas',
                   'Current Supervisor Districts',
                   'Current Police Districts',
                   ]

crime_new_df = crime_df.drop(columns=columns_to_drop)

print("\nNew DataFrame:")
print("-------------------")
print(crime_new_df.dtypes)


Original DataFrame:
-------------------
Incident Datetime                                        object
Incident Date                                            object
Incident Time                                            object
Incident Year                                             int64
Incident Day of Week                                     object
Report Datetime                                          object
Row ID                                                    int64
Incident ID                                               int64
Incident Number                                           int64
CAD Number                                              float64
Report Type Code                                         object
Report Type Description                                  object
Filed Online                                             object
Incident Code                                             int64
Incident Category                                        object


In [8]:
#*************************
#** check new DataFrame **
#*************************
crime_new_df.head(5)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Incident Category,Incident Subcategory,Incident Description,Resolution,Analysis Neighborhood,Latitude,Longitude,Neighborhoods
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,NaN,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,NaN,NaN,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,NaN,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,NaN,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,NaN,NaN


In [9]:
# Remove rows with null values
crime_new_df = crime_new_df.dropna()
crime_new_df.head(5)
num_rows = len(crime_new_df)
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 786835


In [10]:
#*********************************************
#** obtain the unique incident descriptions **
#*********************************************

unique_counts = crime_new_df["Incident Category"].value_counts()
unique_incidents_df = pd.DataFrame(unique_counts.items(), columns=["Unique_Values", "Counts"])
unique_incidents_df

,Unique_Values,Counts
0,Larceny Theft,227727
1,Other Miscellaneous,56346
2,Malicious Mischief,53872
3,Assault,50745
4,Non-Criminal,47792
5,Burglary,45772
6,Motor Vehicle Theft,44600
7,Fraud,26416
8,Recovered Vehicle,24087
9,Warrant,23772


In [11]:
unique_incidents_df.to_csv("incident_categories.csv", index=False)

In [12]:
#********************************************
#** drop categories that aren't of interest**
#** for this project.                      **
#********************************************

keep_categories = ['Larceny Theft',
                   'Malicious Mischief',
                   'Assault',
                   'Burglary',
                   'Motor Vehicle Theft',
                   'Drug Offense',
                   'Robbery',
                   'Missing Person',
                   'Offences Against The Family And Children',
                   'Weapons Offense',
                   'Arson',
                   'Vandalism',
                   'Sex Offense',
                   'Prostitution',
                   'Rape',
                   'Homicide',
                   'Human Trafficking (A), Commercial Sex Acts',
                   'Human Trafficking, Commercial Sex Acts',
                   'Human Trafficking (B), Involuntary Servitude']

#keep_categories = ['Larceny Theft']


In [13]:
num_rows = len(crime_new_df)
print("Number of rows in the DataFrame:", num_rows)
print("-----------------------------------")
crime_new02_df=crime_new_df
crime_new02_df = crime_new_df.loc[crime_new_df['Incident Category'].isin(keep_categories)]
#crime_new02_df.head(5)
num_rows_new = len(crime_new02_df)
print("Number of rows in the DataFrame:", num_rows_new)


Number of rows in the DataFrame: 786835
-----------------------------------
Number of rows in the DataFrame: 503415


In [18]:
unique_counts02 = crime_new02_df["Incident Category"].value_counts()
unique_incidents02_df = pd.DataFrame(unique_counts02.items(), columns=["Unique_Values", "Counts"])
unique_incidents02_df

,Unique_Values,Counts
0,Larceny Theft,227727
1,Malicious Mischief,53872
2,Assault,50745
3,Burglary,45772
4,Motor Vehicle Theft,44600
5,Drug Offense,21096
6,Robbery,18512
7,Missing Person,17893
8,Offences Against The Family And Children,10975
9,Weapons Offense,5768


In [19]:
#**********************************************
# ** export the filtered dataframe to CSV    **
# ** This CSV might enable team members to   **
# ** progress with their parts of the project**
#**********************************************

crime_new02_df.to_csv("crime_new02.csv", index=False)

In [20]:
#*******************************************
#** separate the data into records by year**
#*******************************************

#**************************************************
#** first, count the number of incidents per year**
#**************************************************

year_unique_counts = crime_new02_df["Incident Year"].value_counts()
year_unique_counts_df = pd.DataFrame(year_unique_counts.items(), columns=["Unique_Values", "Counts"])
year_unique_counts_df

,Unique_Values,Counts
0,2018,87225
1,2019,84347
2,2022,82478
3,2023,79050
4,2021,78053
5,2020,69827
6,2024,22435


In [21]:
#***************************************************
#** second, set the seed value and the sample size**
#***************************************************

random_seed = 24      # the last two digits of the current year
sample_size = 1429    #1,429 records per year x 7 years =  10,003 records total

#group the incident data by Incident Year
grouped = crime_new02_df.groupby("Incident Year")

#create an empty DataFrame to store the sampled data
sampled_data = pd.DataFrame()

#now, iterate through each of the groups and sample the data
for year, group_data in grouped:
    sampled_year_data = group_data.sample(n=sample_size, random_state=random_seed)
    sampled_data = pd.concat([sampled_data, sampled_year_data])

# Reset the index of the final sampled DataFrame
sampled_data.reset_index(drop=True, inplace=True)
sampled_data.head(10)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Incident Category,Incident Subcategory,Incident Description,Resolution,Analysis Neighborhood,Latitude,Longitude,Neighborhoods
0,2018/06/15 10:05:00 PM,2018/06/15,22:05,2018,Friday,Missing Person,Missing Person,Found Person,Open or Active,Sunset/Parkside,37.738900,-122.485074,40.0
1,2018/08/24 12:00:00 AM,2018/08/24,00:00,2018,Friday,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Noe Valley,37.742764,-122.428806,84.0
2,2018/08/27 08:10:00 AM,2018/08/27,08:10,2018,Monday,Assault,Aggravated Assault,"Assault, Aggravated, W/ Other Weapon",Open or Active,Visitacion Valley,37.711915,-122.416000,75.0
3,2018/07/20 06:00:00 PM,2018/07/20,18:00,2018,Friday,Larceny Theft,Larceny Theft - Shoplifting,"Theft, Shoplifting, >$950",Open or Active,North Beach,37.805825,-122.411950,99.0
4,2018/03/27 07:00:00 AM,2018/03/27,07:00,2018,Tuesday,Larceny Theft,Larceny Theft - Bicycle,"Theft, Bicycle, >$950",Open or Active,South of Market,37.774516,-122.413769,32.0
5,2018/12/01 04:15:00 PM,2018/12/01,16:15,2018,Saturday,Burglary,Burglary - Other,"Burglary, Non-residential, Unlawful Entry",Open or Active,Mission Bay,37.769866,-122.402404,33.0
6,2018/03/15 04:00:00 PM,2018/03/15,16:00,2018,Thursday,Larceny Theft,Larceny Theft - Other,"Theft, From Person, $200-$950 (other than Pick...",Open or Active,Outer Mission,37.722240,-122.443793,80.0
7,2018/10/23 08:00:00 PM,2018/10/23,20:00,2018,Tuesday,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $50-$200",Open or Active,Financial District/South Beach,37.789264,-122.401375,108.0
8,2018/12/04 04:14:00 PM,2018/12/04,16:14,2018,Tuesday,Assault,Simple Assault,Battery,Open or Active,Outer Mission,37.724588,-122.434870,90.0
9,2018/07/26 02:24:00 PM,2018/07/26,14:24,2018,Thursday,Offences Against The Family And Children,Other,Violation of Restraining Order,Open or Active,Outer Richmond,37.777435,-122.500660,8.0


In [22]:
unique_counts03 = sampled_data["Incident Category"].value_counts()
unique_incidents03_df = pd.DataFrame(unique_counts03.items(), columns=["Unique_Values", "Counts"])
unique_incidents03_df

,Unique_Values,Counts
0,Larceny Theft,4436
1,Malicious Mischief,1054
2,Assault,1052
3,Motor Vehicle Theft,928
4,Burglary,922
5,Drug Offense,445
6,Robbery,352
7,Missing Person,340
8,Offences Against The Family And Children,215
9,Weapons Offense,133


In [23]:
#*********************************************************
#** The sampled data is what we'll use for the project. **
#** Export the sampled data to CSV.                     **
#*********************************************************

sampled_data.to_csv("sample_data_by_year.csv", index=False)

In [24]:
#*********************************
#** PART TWO: DATABASE CREATION **
#*********************************

In [25]:
#******************************
#** 1. connect to MongoDB    **
#** 2. create the database   **
#** 3.  add the collection   **
#******************************

client = MongoClient('mongodb://localhost:27017/')
db = client["crime_db"]      #this creates a Mongo database named "crime_db"
db.drop_collection("incidents") #this drops the collection if it already exists - so records aren't duplicated.
collection = db["incidents"] #this creates a collection named "incidents" within the Mongo database

In [26]:
#**********************************
#** import the source data to    **
#** the collection created in the**
#** previous step                **
#**********************************

with open('sample_data_by_year.csv','r') as file:
    reader = csv.DictReader(file)
    data = []
    for row in reader:
        data.append(row)
        
collection.insert_many(data)
print("sample data imported into the MongoDB collection successfully!")

sample data imported into the MongoDB collection successfully!
